In [84]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')

import os
import re
import gc

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', 200)

from tqdm.notebook import tqdm

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, minmax_scale,OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import auc, roc_auc_score

from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [85]:
train_data = pd.read_csv('/home/c506plus/wq/sohu/sohuBaseline2022REC/sohuBaseline2022/data/train-dataset.csv')
test_data = pd.read_csv('/home/c506plus/wq/sohu/sohuBaseline2022REC/sohuBaseline2022/data/test-dataset.csv')
senti_feats = pd.read_csv('/home/c506plus/wq/sohu/sohuBaseline2022REC/sohuBaseline2022/data/senti_feats.csv')

In [86]:
data=pd.concat([train_data,test_data],axis=0,ignore_index=True)

In [87]:
data = data.merge(senti_feats, left_on='itemId', right_on='id', how='left')

In [88]:
data.head()

,sampleId,label,pvId,suv,itemId,userSeq,logTs,operator,browserType,deviceType,osType,province,city,testSampleId,id,senti_0_max,senti_0_min,senti_0_mean,senti_0_std,senti_1_max,senti_1_min,senti_1_mean,senti_1_std,senti_2_max,senti_2_min,senti_2_mean,senti_2_std,senti_3_max,senti_3_min,senti_3_mean,senti_3_std,senti_4_max,senti_4_min,senti_4_mean,senti_4_std,entity_count
0,1.0,0.0,1420332726637uyIZR2O,1581173072343wnwm3q,10092752,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,10092752.0,2.866753,-1.794564,-1.100593,1.211579,3.234082,-1.485561,-0.261668,1.257977,8.180731,0.310928,6.948261,2.189150,-0.547445,-2.442261,-1.264404,0.423758,-2.102074,-3.930464,-2.909978,0.507412,14.0
1,2.0,0.0,1420332726637uyIZR2O,1581173072343wnwm3q,10085565,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,10085565.0,2.974319,-3.392552,-0.820591,1.786052,2.585300,-1.316303,0.771271,1.414772,7.788972,-0.762745,4.465163,2.853193,4.710341,-2.759928,-0.716414,2.345446,0.100399,-3.856450,-3.013716,1.218353,10.0
2,3.0,0.0,1420332726637uyIZR2O,1581173072343wnwm3q,10105937,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,10105937.0,0.467619,-2.308136,-1.275848,1.054758,1.601117,-1.227540,-0.135687,1.332704,8.180069,0.999190,6.262783,3.042464,1.292194,-1.546283,-0.718128,1.020301,-2.276604,-2.806367,-2.582355,0.229076,6.0
3,4.0,0.0,1420332726637uyIZR2O,1581173072343wnwm3q,10082274,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,10082274.0,-0.989341,-3.975087,-2.512234,1.039144,4.834974,-1.836809,0.471163,1.843924,8.193871,-1.399417,4.074359,3.097720,3.027706,-2.311181,1.069665,1.559377,-1.574592,-4.144028,-2.672037,0.650114,19.0
4,5.0,0.0,1420332726637uyIZR2O,1581173072343wnwm3q,10083446,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140,NaN,10083446.0,-0.647296,-3.208978,-1.594158,0.601857,2.747163,-0.909461,0.999508,1.460980,8.117958,4.243027,6.062799,1.383080,1.978340,-2.392845,-1.252125,1.317016,-2.076510,-4.347492,-3.532305,0.770072,12.0


In [89]:
#时间特征处理
data['ts_datetime']=data['logTs']+8*60*60*1000
data['ts_datetime']=pd.to_datetime(data['ts_datetime'],unit='ms')
data['day']=data['ts_datetime'].dt.day
data['hour']=data['ts_datetime'].dt.hour
data['minute']=data['ts_datetime'].dt.minute
data['minute10']=(data['minute'] // 10) * 10
data['hourl'] = data['day'] * 24 + data['hour']
data['hourl'] = data['hourl'] - data['hourl'].min()


In [90]:
#构建交叉特征
#省份和城市之间交叉特征
data['prov_city']=data['province']+data['city']
#操作系统，运营商，操作系统，浏览器类型交叉特征
data['device_os']=data['deviceType']+data['osType']
data['opera_browser']=data['operator']+data['browserType']
data['device_os_opera_browser']=data['device_os']+data['opera_browser']
#用户和pvId之间交叉
data['suv_pvId']=data['suv']+'_'+data['pvId']
#在不同的地方和城市你可能会刷不同的新闻
data['prov_city_device_os']=data['prov_city']+data['device_os']
data['prov_city_opera_browser']=data['prov_city']+data['opera_browser']


In [91]:
#用户距离上一次点击文章的时间差，时间差越大说明用户可能对该文章感兴趣
data['suv_to_lasttime']=data.groupby('suv')['logTs'].diff().values

In [92]:
#对object特征进行frequency_encoder处理
for feat in ['itemId','suv','pvId','logTs','operator','browserType',
             'deviceType','osType','province','city']:
    data[feat+'_cnt']=data[feat].map(data[feat].value_counts())


In [93]:
# #train_feature enginering
for feat in [ 'pvId','suv','itemId','operator', 'browserType', 
             'deviceType', 'osType', 'province', 'city','suv_pvId']:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# for feat in  [ 'pvId', 'suv', 'itemId']:
#     ohe=OneHotEncoder()
#     train_data[feat] = ohe.fit_transform(train_data[[feat]])

data['pvid_count'] =data.groupby('pvId')['itemId'].transform('count')
data['pvid_item_nunique'] = data.groupby('pvId')['itemId'].transform('nunique')
data['pvid_item_count'] =data.groupby('pvId')['itemId'].transform('count')
data['pvid_suv_nunique'] = data.groupby('pvId')['suv'].transform('nunique')
data['pvid_suv_count'] = data.groupby('pvId')['suv'].transform('count')

data['item_count'] = data.groupby('itemId')['itemId'].transform('count')
data['item_suv_nunique'] = data.groupby('pvId')['suv'].transform('nunique')
data['suv_click_count']=data.groupby(['suv','itemId'])['logTs'].transform('count')

In [94]:
target=['label']

In [95]:
dense_features = ['day','hour','minute','minute10','hourl','itemId_cnt',
                  'suv_cnt','pvId_cnt','logTs_cnt','operator_cnt','browserType_cnt',
                  'deviceType_cnt','osType_cnt','province_cnt','city_cnt',
                 'pvid_count','pvid_item_nunique','pvid_item_count',
                 'pvid_suv_nunique','pvid_item_count','pvid_suv_nunique',
                 'pvid_suv_count','item_count','item_suv_nunique','suv_click_count',
                 'pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique',
                  'item_count', 'item_suv_nunique',
                   'senti_0_max', 'senti_0_min', 'senti_0_mean',
                   'senti_0_std', 'senti_1_max', 'senti_1_min', 'senti_1_mean',
                   'senti_1_std', 'senti_2_max', 'senti_2_min', 'senti_2_mean',
                   'senti_2_std', 'senti_3_max', 'senti_3_min', 'senti_3_mean',
                   'senti_3_std', 'senti_4_max', 'senti_4_min', 'senti_4_mean',
                    'senti_4_std', 'entity_count']

In [96]:
#data_dense
data[dense_features] = data[dense_features].fillna(0, )
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [97]:
data.drop(['userSeq','logTs','ts_datetime'], axis=1, inplace=True)


In [98]:
train_data.head()

,sampleId,label,pvId,suv,itemId,userSeq,logTs,operator,browserType,deviceType,osType,province,city
0,1,0,1420332726637uyIZR2O,1581173072343wnwm3q,10092752,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
1,2,0,1420332726637uyIZR2O,1581173072343wnwm3q,10085565,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
2,3,0,1420332726637uyIZR2O,1581173072343wnwm3q,10105937,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
3,4,0,1420332726637uyIZR2O,1581173072343wnwm3q,10082274,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
4,5,0,1420332726637uyIZR2O,1581173072343wnwm3q,10083446,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140


In [99]:
train_data = data[data['label'].notna()]
test_data = data[data['label'].isna()]

train_data.drop(['testSampleId'], axis=1, inplace=True)
test_data.drop(['sampleId', 'label'], axis=1, inplace=True)

print(train_data.shape, test_data.shape)

(4233804, 64) (1422218, 63)


In [100]:
train=train_data.copy()

In [101]:
train_data.drop(['label'], axis=1, inplace=True)

In [102]:
# # create dataset for lightgbm
# lgb_train = lgb.Dataset(train,train_data[target])
# lgb_test=lgb.Dataset(test)
# # lgb_eval = lgb.Dataset(train, train_data[target], reference=lgb_train)

# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': {'binary_logloss'},
#     'num_leaves': 64,
#     'num_trees': 10,
#     'learning_rate': 0.01,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }

# # number of leaves,will be used in feature transformation
# num_leaf = 64

# print('Start training...')
# # train
# gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=100,
#                 valid_sets=lgb_test)

In [103]:
from lightgbm import LGBMClassifier

lgbclassifier=LGBMClassifier(boosting_type='gbdt',
                            num_leaves=31,
                            max_depth=7,
                            n_estimators=300, 
                            objective='binary',
                            learning_rate=0.1,
                            importance_type='gain',
                            metric='AUC',
                            verbose=10)
lgbclassifier.fit(train_data,train[target])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 816351, number of negative: 3417453
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.860539
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.182300
[LightGBM] [Debug] init for col-wise cost 0.167342 seconds, init for row-wise cost 0.470739 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.237230 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 9655
[LightGBM] [Info] Number of data points in the train set: 4233804, number of used features: 63
[Light

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Traine

LGBMClassifier(importance_type='gain', max_depth=7, metric='AUC',
               n_estimators=300, objective='binary', verbose=10)

In [104]:
import pandas as pd
df=lgbclassifier.predict_proba(test_data)
df = pd.DataFrame(df)
df

,0,1
0,0.573977,0.426023
1,0.596627,0.403373
2,0.221885,0.778115
3,0.557765,0.442235
4,0.331280,0.668720
...,...,...
1422213,0.892166,0.107834
1422214,0.378762,0.621238
1422215,0.895508,0.104492
1422216,0.400839,0.599161


In [105]:
sub=pd.read_csv('/home/c506plus/wq/sohu/submission/section2.txt', sep='\t')

In [106]:
sub['result']=df[1]

In [107]:
sub.head()

,Id,result
0,5000001,0.426023
1,5000002,0.403373
2,5000003,0.778115
3,5000004,0.442235
4,5000005,0.668720


In [108]:
sub.to_csv('4_27_4_submission.txt',sep='\t', index=False)